In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import auc, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb


In [5]:
#subs = pd.read_csv("data/auctions.csv.gzip",dtype = {"auction_type_id": np.float16, "platform": np.int8, "ref_type_id": np.int8, "source_id": np.int8})
#evts = pd.read_csv("data/events.csv.gzip", dtype = {"event_id": np.int16, "application_id": np.int16})
#clks = pd.read_csv("data/clicks.csv.gzip", dtype = {"advertiser_id": np.int8, "action_id": np.float16, "source_id": np.int8, "carrier_id": np.float16, "brand": np.float16, "touchX": np.float16, "touchY": np.float16})

In [22]:
import gzip
def ggzip(source_filepath, dest_filepath, block_size=65536):
    with gzip.open(dest_filepath, 'wb') as d_file, \
            open(source_filepath, 'rb') as s_file:
        for i in range(block_size):
            line = s_file.readline()
            if not line:
                break
            else:
                d_file.write(line)
        #d_file.write(block)
        
ggzip("data/auctions.csv.gzip", "data/auctions_reducido.csv.gzip", 100000)

In [23]:
auct = pd.read_csv("data/auctions_reducido.csv.gzip", compression = 'gzip' )

In [24]:
auct

,date,device_id,ref_type_id,source_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0
5,2019-04-23 18:58:02.675804,6707090658317158573,1,0
6,2019-04-23 18:58:02.848212,8869722088125970841,1,0
7,2019-04-23 18:58:03.048872,7445213948764639634,1,0
8,2019-04-23 18:58:03.285788,2932617030932207332,1,0
9,2019-04-23 18:58:03.532112,6405811806780450397,7,0
